In [53]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import interpolation as inter
from PIL import Image as im
import re
# from segmentation.utilities import projection, save_image
# from segmentation.preprocessing import binary_otsus, deskew
from segmentation.segmentation import line_horizontal_projection, word_vertical_projection
from segmentation.character_segmentation import segment
from tensorflow.keras.models import load_model
import arabic_reshaper
from bidi.algorithm import get_display
from Preprocessing.preprocessing import otsu_threshold, crop_on_borders
from PIL import Image
import pickle

In [54]:
with open("chars.pkl", "rb") as file:
    chars_set = pickle.load(file)

print("Dictionary loaded:", chars_set)

Dictionary loaded: {0: 'أ', 1: 'ب', 2: 'ت', 3: 'ث', 4: 'ج', 5: 'ح', 6: 'خ', 7: 'د', 8: 'ذ', 9: 'ر', 10: 'ز', 11: 'س', 12: 'ش', 13: 'ص', 14: 'صلى', 15: 'ض', 16: 'ط', 17: 'ظ', 18: 'ع', 19: 'غ', 20: 'ف', 21: 'ق', 22: 'ك', 23: 'ل', 24: 'م', 25: 'ن', 26: 'ه', 27: 'و', 28: 'ي'}


### **1 - Reading Function:**

In [55]:
def read_image(path):
    img = cv.imread(path)  # Read the image
    if img is not None:
        return img  # Return the image array if successfully read
    else:
        print(f"Error reading image: {path}")
        return None  # Return None for failed reads

### **2 - Segementation Function:**

In [56]:
def Segment(img):

    words_chars = []
    lines = line_horizontal_projection(img)
    for line in lines:
        words = word_vertical_projection(line)
        for word in words :
            cr = segment(line, word)
            words_chars.append(cr)

    return words_chars

### **3 - Loading Model & Predicting:**

In [57]:
path_model = "model.h5"
model = load_model(path_model)
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 58, 58, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 58, 58, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 29, 29, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 29, 29, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 27, 27, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 27, 27, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 11, 11, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 11, 11, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 5, 5, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 5, 5, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │       409,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 29)             │         1,885 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 513,951 (1.96 MB)

 Trainable params: 513,245 (1.96 MB)

 Non-trainable params: 704 (2.75 KB)

 Optimizer params: 2 (12.00 B)

In [58]:
# Just some testing
word = ''
lis = ["ع", "ر", "ب", "ي"]
for i in lis:
    word +=i
word += ' '
word+= lis[0]
print("Original Text:", word)


Original Text: عربي ع


#### **Testing & Predicting :** 

In [59]:
image_path = 'Dataset/img_1.png'
# img = preprocess(image_path)
img = read_image(image_path)
words_ch = Segment(img)
text = ''
j = 0
for ch in words_ch:
    word = ''
    for c in ch :
        img = Image.fromarray(c)
        crop_pars = crop_on_borders(img)
        cropped = np.array(img.crop(crop_pars).resize((60,60)).convert("L"))
        temp = np.array([cropped])
        res = model.predict(temp)
        char = chars_set[np.argmax(res)]
        word += char
        print(char)
        j += 1
    text += word + ' '

c:\Users\s_bou\Downloads\DL_OCR_Project\Preprocessing\preprocessing.py:23: RuntimeWarning: invalid value encountered in scalar divide
  u0 = np.sum(np.arange(i) * hist[:i]) / (w0 * total_pixels)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step
و
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
ع
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
ن
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
أ
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
م
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
ي
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
ر
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
أ
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
ل
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
م
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
ض
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
م
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
ن
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
ي
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
ن
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
ن
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
أ
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
ب
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
ي
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
ح
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
ق
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
ص
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
ن
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
ع
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/st

In [60]:
print('Predicted text : \n')
print(text)

Predicted text : 

وعن أمير ألمضمنينن أبي حقصن عو بنن ألخطأبد بنن نفيل بنن دعبد ألعزين بنن ريأح بن قرط بن رزأح بن عس ين بن ضيه أين عألبه ألقرتب أعوين رضن ألله 
